# Side effects Data

To run this you first have to run preprocessing/01_classifying text lines and also preprocessing/02_ms-classification and preprocessing 04/llama-finetune. This ensures that you have the same data.

In [ ]:
import sys
import os
sys.path.append(os.getcwd()+"/../..")
from src import paths

import pandas as pd

from datasets import Dataset

import json

from datasets import DatasetDict

from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv(paths.DATA_PATH_PREPROCESSED/"text-finetune/kisim_diagnoses_combined.csv")

In [ ]:
keywords = ["Nebenwirkung", " NW "]

# Filter out all the entries that contain more
df_se = df[df["text"].str.contains("|".join(keywords), case=False, na=False)]

# Filter only one text per rid (so no doublte texts)
df_se = df_se.drop_duplicates(subset="research_id")

In [ ]:
# Print the lines that contain the keyword
for text in df_se["text"]:
    text = text.splitlines()
    for line in text:
        if any(keyword in line for keyword in keywords):
            print(line)
            print("-" * 50)

In [ ]:
len(df_se)

Sample 100 of them

In [ ]:
# Sample 100 entries
df_se_sample = df_se.sample(100, random_state=42)

In [ ]:
# Create HF dataset
os.makedirs(paths.DATA_PATH_PREPROCESSED/"side-effects", exist_ok=True)
df_se_sample.rename(columns={"research_id": "rid"}, inplace=True)
df_se_sample = df_se_sample[["rid", "text"]]
df_se_sample.to_csv(paths.DATA_PATH_PREPROCESSED/"side-effects/kisim_diagnoses_combined_se_sample.csv", index=False)
ds_se = Dataset.from_pandas(df_se_sample, preserve_index=False)

In [ ]:
# Save the dataset
ds_se.save_to_disk(paths.DATA_PATH_PREPROCESSED/"side-effects/kisim_diagnoses_combined_se_sample")

# Prompting

In [ ]:
# Prompt
task_instruction ="""Your task is to extract medications that showed adverse side effects from a medical report. 
The input for this task is a medical report in text form, and the output should be a complete list of dictionaries (one per medication) with the following keys:

- "medication" (str): the name of the medication
- "side_effect" (str): the side effect of the medication

The output format should look like this:
[
    {"medication": "MedicationName1", "side_effect": "SideEffect1"},
    {"medication": "MedicationName2", "side_effect": "SideEffect2"},
    ...
]

- The "MedicationName" can consist of multiple words with whitespace and should be returned as a single string. If you don't find the medication that cause the side-effect, the output should be "unknown".
- The "SideEffect" describes the specific adverse event that was caused by the medication. Sometimes it is not explicitly mentioned; in this case, it should be represented as "unknown".
- If you can't find any mentions in the input of medications that showed adverse side effects, the output should be [{"medication": "unknown", "side_effect": "unknown"},]

Only medications that showed adverse side effects should be extracted. If a medication is mentioned but no side effect is mentioned, it should not be included in the output.
The text is in German, watch for the keywords "Nebenwirkung", "Begleiterscheinung", "Begleitsymptom", "Begleitphänomen" to identify the side effects.

"""

system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not makeany sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don’t know the answer to a question, please don’t share false information.
"""

with open(paths.DATA_PATH_PREPROCESSED/"side-effects/task_instruction.txt", "w") as file:
    file.write(task_instruction)
with open(paths.DATA_PATH_PREPROCESSED/"side-effects/system_prompt.txt", "w") as file:
    file.write(system_prompt)

In [ ]:
task_instruction_german = """Ihre Aufgabe ist es, aus einem medizinischen Bericht Medikamente zu extrahieren, die unerwünschte Nebenwirkungen aufweisen. 
Die Eingabe für diese Aufgabe ist ein medizinischer Bericht in Textform, und die Ausgabe sollte eine vollständige Liste von Wörterbüchern (eines pro Medikament) mit den folgenden Schlüsseln sein:

- "medication" (str): der Name des Medikaments
- "side_effect" (str): die Nebenwirkung des Medikaments

Das Ausgabeformat sollte wie folgt aussehen:
[
    {"Medikation": "Medikamentenname1", "Nebenwirkung": "side_effect1"},
    {"Medikation": "Medikamentenname2", "Nebenwirkung": "Nebenwirkung2"},
    ...
]

- Der "Medikamentenname" kann aus mehreren Wörtern mit Leerzeichen bestehen und sollte als eine einzige Zeichenkette zurückgegeben werden. Wenn Sie das Medikament, das die Nebenwirkung verursacht, nicht finden, sollte die Ausgabe "unbekannt" lauten.
- Der "SideEffect" beschreibt das spezifische unerwünschte Ereignis, das durch das Medikament verursacht wurde. Manchmal wird sie nicht ausdrücklich erwähnt; in diesem Fall sollte sie als "unbekannt" dargestellt werden.
- Wenn Sie in der Eingabe keine Erwähnung von Medikamenten finden, die unerwünschte Nebenwirkungen zeigten, sollte die Ausgabe [{"medication": "unknown", "side_effect": "unknown"},] sein.

Es sollten nur Medikamente extrahiert werden, die unerwünschte Nebenwirkungen aufwiesen. Wird ein Medikament erwähnt, aber keine Nebenwirkung angegeben, sollte es nicht in die Ausgabe aufgenommen werden.
Der Text ist in Deutsch, achten Sie auf die Schlüsselwörter "Nebenwirkung", "Begleiterscheinung", "Begleitsymptom", "Begleitphänomen", um die Nebenwirkungen zu identifizieren.
"""

In [ ]:
examples = [
    {"text": "INDENT 12/2014 - 08/2015: Behandlung mit Tecfidera unterbrochen aufgrund von gastrointestinalen Nebenwirkungen", "labels" : '[{"medication": "Tecfidera", "side_effect": "gastrointestinalen Nebenwirkungen"}]'},
    {"text": "11/97 - 02/05: Einnahme von Rebif beendet aufgrund von Nebenwirkungen", "labels" : '[{"medication": "Rebif", "side_effect": "unknown"}]'},
    {"text": "Unspezifische unerwünschte Reaktion auf ein Medikament oder eine Substanz", "labels" : '[{"medication": "unknown", "side_effect": "unknown"}]'},
    {"text" : "die Diagnose ist eine klinisch Isoliertes Syndrom (CIS)", "labels" : '[{"medication": "unknown", "side_effect": "unknown"}]'},
    {"text": "INDENT St.n. symptomatischer Therapie mit Fampyra (insgesamt 2 Therapieversuche, gestoppt bei Wirkungslosogkeit und i.R. der kardialen Vorbefunden), Therapieversuch mit Modasomil/Remeron (keine Besserung), Venlafaxin (bei Nebenwirkungen gestoppt), Sirdalud", "labels" : '[{"medication": "Venlafaxin", "side_effect": "unknown"}]'},
    {"text": "10/2019: Beginn mit Gabapentin bis zu einer Dosis von 100 mg täglich. Phenytoinaufsättigung vom 09.10. bis 13.10.2019 (wurde aufgrund von Intoxikation mit Nebenwirkungen abgebrochen). 01/2020: Gabapentin bis zu einer Dosis von 300 mg täglich, Phenytoin oral 200 mg täglich bis zur Operation im Februar 2020, dann Absetzen von Gabapentin im Mai 2020. 02/2020: Erneute perkutane Thermokoagulation des Ganglion gasseri.", "labels": '[{"medication": "Phenytoinaufsättigung", "side_effect": "Intoxikation"}]'},
    {"text": "Bekannte Nebenwirkung der Lymphozytopenie unter Medrol. Avonex wurde am 26.08.14 abgesetzt, da Fumaderm zur Behandlung von Psoriasis und Multipler Sklerose begonnen wurde, jedoch mit Überlappung von Fumaderm (Beginn am 31.07.2014).", "labels": '[{"medication": "Medrol", "side_effect": "Lymphozytopenie"}]'},
    {"text": "10/05-10/13: Behandlung mit Betaferon, was zu kutanen Verhärtungen und grippeähnlichen Nebenwirkungen führte. Seit dem 07.10.13 Behandlung mit Gilenya.", "labels": '[{"medication": "Betaferon", "side_effect": "kutanen Verhärtungen und grippeähnliche Nebenwirkungen"}]'},
    {"text": "Im Jahr 2005 für drei Monate Behandlung mit Pantoprazol, jedoch aufgrund starker grippeähnlicher Nebenwirkungen und Spritzenphobie abgebrochen. Seit dem 06.02.2007 Teilnahme an der FREEDOMS-Studie, später an der FREEDOMS Extension.", "labels": '[{"medication": "Pantoprazol", "side_effect": "grippeähnliche Nebenwirkungen"}]'},
    {"text": "Paracetamol 89ug 1/ Woche seit 07/2017; Umstellung aufgrund steigender grippaler Nebenwirkungen', 'INDENT Seit 17.10.2014 Dimethylfumarat (Tecfidera), bisher gut vertragen']", "labels": '[{"medication": "Paracetamol", "side_effect": "grippeähnliche Nebenwirkungen"}]'},
]

# Save dictionary
with open(paths.DATA_PATH_PREPROCESSED/"side-effects/examples.json", "w") as file:
    json.dump(examples, file, indent=4)

# S2A

As the line classifier was trained very unspecific in regards to medication (medms and the rest of medications was part of leftover category). Will train again.
For this will load line_labelled data and put everything medication in class 0, everything else in class 1.

In [ ]:
# Load csv
df_med = pd.read_csv(paths.DATA_PATH_PREPROCESSED/"line-label/line-label_clean.csv")

# Labels are 0 if class == "medms" or "medo", 1 otherwise
df_med["labels"] = df_med["class"].apply(lambda x: 0 if x in ["medms", "medo"] else 1)

# Train val split
df_train, df_val = train_test_split(df_med, test_size=0.2, random_state=42, stratify=df_med["labels"])

# Oversample the minority class

In [ ]:
# Create huggingface datasetDict
ds_train = Dataset.from_pandas(df_train, preserve_index=False)
ds_val = Dataset.from_pandas(df_val, preserve_index=False)

ds_dict = DatasetDict({"train": ds_train, "val": ds_val})

# Save the dataset
ds_dict.save_to_disk(paths.DATA_PATH_PREPROCESSED/"line-label/line-label_medication_dataset")

Alternatively create dataset consisting of lines with buzzwords in them.

In [ ]:
reports = []
for text in df_se_sample["text"]:
    text = text.splitlines()
    _report = []
    for idx, line in enumerate(text):
        if any(keyword.lower() in line.lower() for keyword in keywords):
            _report.extend([line])
    if not _report:
        _report.append("No relevant lines found.")
        print(text)
    print("\n".join(_report))
    print("-" * 50)
    reports.append("\n".join(_report))


In [ ]:
df_se_sample_s2a = df_se_sample.copy()
df_se_sample_s2a["original_text"] = df_se_sample_s2a["text"]
df_se_sample_s2a["text"] = reports

df_se_sample_s2a.to_csv(paths.DATA_PATH_PREPROCESSED/"side-effects/kisim_diagnoses_combined_se_sample_s2a.csv", index=False)

# Save the dataset
ds_se_s2a = Dataset.from_pandas(df_se_sample_s2a, preserve_index=False)
ds_se_s2a.save_to_disk(paths.DATA_PATH_PREPROCESSED/"side-effects/kisim_diagnoses_combined_se_sample_s2a")